# Load packages

In [8]:
import geopandas as gpd
import pandas as pd
import numpy as np
from rasterstats import zonal_stats
import rasterio
import rioxarray
import xarray

In [9]:
# Define directories
dir_shp = 'M:/MENA/GEO/Boundaries/'
dir_in = 'M:/MENA/GEO/Hazards/TEMP/raw/'
dir_out = 'M:/MENA/GEO/Hazards/TEMP/final/'

# Define output FILE
VAR = 'TEMP_'
YEAR_LIST = [2012,2013,2014,2015,2016,2017,2018,2019,2020]
MONTH_LIST = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
FILE_OUT = 'ADM2_' + VAR + str(min(YEAR_LIST)) + '_' + str(max(YEAR_LIST)) + '.csv'

In [10]:
%matplotlib inline

In [11]:
# Load shapefile
MENA_shp = gpd.read_file(dir_shp + 'MENA_ADM2.shp')

MENA_shp.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [12]:
df_full = pd.DataFrame(MENA_shp['ID_ADM'])

for YEAR in YEAR_LIST:
    for MONTH in MONTH_LIST:
        with rasterio.open(dir_in + 'TEMP_' + str(YEAR)+ '_' + str(MONTH) + '.tif') as raster:
            array = raster.read(1)
            affine = raster.transform
            nodata = raster.nodata

        MENA_shp.to_crs(raster.crs, inplace = True)

        # Extract raster by polygon
        dict_temp = zonal_stats(MENA_shp, array, affine = affine, nodata = nodata, 
                                interpolate = 'bilinear', all_touched = True,
                                stats = ['mean'])
        df_temp = pd.DataFrame(dict_temp)
        df_temp = df_temp.add_prefix(VAR)

        df_temp = df_temp.add_suffix('_' + str(YEAR)+ str(MONTH))
        df_temp.insert(0, 'ID_ADM', MENA_shp['ID_ADM'])
    
        df_full = df_full.merge(df_temp, how = 'left', on = 'ID_ADM')

In [7]:
df_full

,ID_ADM,TEMP_mean_20121
0,37631,14785.164827
1,37632,14886.135379
2,37633,14746.738153
3,37634,14868.440343
4,37635,14936.209559
...,...,...
2932,15561,14438.385569
2933,15562,14624.736830
2934,15563,14754.903207
2935,15564,14593.421126


In [ ]:
# Assuming you've already defined and populated df_full
df_full.to_csv("M:/MENA/GEO/Hazards/TEMP/final/TEMP_2012_2020.csv", index=False)